In [15]:
import math
import string

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset, IterableDataset

import torchdata
import torchtext

import re
import glob
import warnings
import unicodedata
import string

In [16]:
def get_files(path):
    return glob.glob(path)

def unicodeToAscii(word):
    return ''.join(c for c in unicodedata.normalize('NFD', word)
        if unicodedata.category(c) != 'Mn' and c in all_letters
    )

def readLine(fname):
    file_name = re.findall(r'/data/names/(.*).txt', fname)[0]
    with open(fname,encoding='utf-8') as file:
        return file_name,[unicodeToAscii(line.strip()) for line in file]
    
def getCategory(path):
    files = get_files(path)
    category = [re.findall(r'/data/names/(.*).txt', file_name)[0] for file_name in files]
    return category

def getCatToName(path):
    files = get_files(path)
    cats = []
    catToName = {}
    for file in files:
        cat, name = readLine(file)
        cats.append(cat)
        catToName[cat] = name
    return cats, catToName

In [63]:
all_letters = string.ascii_letters + ".,';-" + '<EOS>'

In [18]:
### Label
path = './data/names/*.txt'
### Lang to Name
all_cat, catToLine = getCatToName(path)

In [19]:
class RNN_GEN_NAME(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_categories):
        super(RNN_GEN_NAME, self).__init__()
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.n_categories = n_categories
        
        self.inputToHidden = nn.Linear(input_size+hidden_size+n_categories, hidden_size)
        self.inputToOutput = nn.Linear(input_size+hidden_size+n_categories, output_size)
        self.outToOut = nn.Linear(hidden_size+output_size, output_size)
        
        self.dropOut = nn.Dropout(0.3)
        self.act1 = nn.LeakyReLU()
        self.act2 = nn.LeakyReLU()
        self.softMax = nn.LogSoftmax()
        
    def forward(self, category, inp, hid):
        input_comb = torch.cat((category,inp,hid), 1)
        
        hidden = self.inputToHidden(input_comb)
        output = self.inputToOutput(input_comb)
        
        output_combined = torch.cat((hidden, output), 1)
        
        output = self.outToOut(output_combined)
        output = self.dropOut(output)
        output = self.softMax(output)
        
        return output, hidden
    
    def initHidden(self, hidden):
        return torch.zeros(1, hidden)

In [80]:
import random, math

def randomChoice(listToChoice):
    return listToChoice[random.randint(0, len(listToChoice)-1)]

def randomPair(catList, lineList):
    return randomChoice(catList), randomChoice(lineList)

def lineToTensor(line, letters):
    tensor = torch.zeros([len(line), len(letters)])
    
    for i, l in enumerate(line):
        l_to_index = getIndex(letters, l)
        tensor[i][l_to_index] = 1
    
    print(tensor)

def randomExample(catList, lineList, oheCat):
    cat, line = randomPair(catList, lineList)
    catTensor = oheCat[cat]
    lineTensor = lineToTensor(line)

def catOhe(cat):
    lenCat = len(cat)
    tensor = torch.zeros([lenCat, lenCat])
    catToTensor = {}
    for i in range(lenCat):
        tensor[i][i] = 1
        catToTensor[cat[i]] = tensor[i]
    return catToTensor, tensor
    
    
def getIndex(listToSearch, toSearch):
    index = math.inf
    lenList = len(listToSearch)
    
    for i in range(lenList):
        if listToSearch[i] == toSearch:
            index = i
            break
    
    if index >= 0:
        return index 

In [76]:
li = getIndex(all_cat, 'Arabic')

In [77]:
print(li)

0


In [78]:
lineToTensor('ABC', all_letters)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])


In [81]:
oheTensor, tensor = catOhe(all_cat)

In [83]:
print(tensor)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0.,